# Cleaning Ingredients for Bag of Words

## Frequency of Words

In [25]:
vocabulary = nltk.FreqDist()

for ingredients in df['ingredients_cleaned']:
    str1 = ""
#     print(ingredients)
    for i in ingredients:
        str1 += " "+i 
    ingredients = str1.split()
    vocabulary.update(ingredients)
    
for word, frequency in vocabulary.most_common(1000):
    print(f'{word};{frequency}')

In [23]:
vocabulary['?']

0

## Ceaning Ingredients

In [19]:
import unidecode
import ast
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re


def ingredient_cleaner(ingreds):
    lemmatizer = WordNetLemmatizer()
    words_to_remove = ["as", "required", "chopped", "medium", "fresh", "boiled", "peeled", "and", "finely", "a", "pinch",                    
                       "slice", "crushed", "fillet", "sliced", "few", "strand", "browned", "loaf", "piece", "into", "cube", 
                       "garnishing", "tablespo", "drop", "shaving", "drained", "grated", "gram", "soaked", "roughly", "blanched",
                        "scraped", "ice", "large", "halved", "washed", "seeded", "in", "deveined", "diced", "slivered", "shelled",
                       "quartered", "bulb", "overnight", "on", "pearl", "desiccated", "the", "at", "from", "cup", "one", "two", 
                       "three", "four", "five", "six", "seven", "eight", "nine", "ten", "plenty", "of", "", " "]
    
    if isinstance(ingreds, list):
        items = ingreds
    else:
        items = ast.literal_eval(ingreds)
       
    # converting all ingredients to unicode  
    items = [unidecode.unidecode(word) for word in items]
    
    # converting all ingredients to lower case
    items = [word.lower() for word in items]        
    
    # getting rid of all information beyond a comma
    items = [word.split(",")[0] for word in items]
    
    # deleting all punctuations
    items = [word.replace("(", "") for word in items]
    items = [word.replace(")", "") for word in items]
    items = [word.replace("/", " ") for word in items]
    items = [word.replace("-", " ") for word in items]
    items = [word.replace("+", " ") for word in items]
    items = [word.replace("?", " ") for word in items]
    
    # getting rid of all information beyond any number
    items = [re.split(r'(\d+)', word)[0] for word in items]
    
    # getting rid of any information beyond to, for, cut and with
    items = [word.split(" to")[0] for word in items]
    items = [word.split("for")[0] for word in items]
    items = [word.split("cut")[0] for word in items]
    items = [word.split(" with")[0] for word in items]
    
    # replace double spaces with single space
    items = [word.replace("  ", " ") for word in items]
                          
    # delete any ingredient which contains oil, salt, butter, sugar or lemon
    items = [word for word in items if word.find("oil") == -1]
    items = [word for word in items if word.find("salt") == -1]
    items = [word for word in items if word.find("butter") == -1]
    items = [word for word in items if word.find("sugar") == -1]
    items = [word for word in items if word.find("lemon") == -1]
    
    items_lemmatized = []
    for i in range(0, len(items)):
        to_lemma = items[i].split(' ')
        # lemmatizing words
        word_lemmatized = [lemmatizer.lemmatize(word) for word in to_lemma]
        # removing words in words_to_remove
        word_cleaned = ' '.join([word for word in word_lemmatized if word not in words_to_remove])
        items_lemmatized.append(word_cleaned.lstrip().rstrip())
    items_lemmatized = [word for word in items_lemmatized if word != '']
    
    return items_lemmatized   

In [20]:
# importing scrubbed data
df = pd.read_csv('df_scrubbed_final.csv')
# dropping rows with missing values
df = df.dropna()
# applying ingredient_cleaner to add column "ingredients_cleaned"
df['ingredients_cleaned'] = df['ingredients'].apply(lambda x: ingredient_cleaner(x))
df = df.iloc[: , 1:]

In [21]:
df.head()

,recipe_urls,recipe_name,ingredients,prep_time,cook_time,serve,level_of_cooking,taste,ingredients_cleaned
0,https://www.sanjeevkapoor.com/Recipe/Taazi-Hal...,Taazi Haldi Aur Gajar Ki Sabzi,"['Fresh turmeric (taazi haldi) , chopped 1 1/2...",11-15,21-25,4,Easy,Spicy,"[turmeric taazi haldi, carrot, ghee, cumin see..."
1,https://www.sanjeevkapoor.com/Recipe/Jackfruit...,Jackfruit Seed Sambhar,"['Ripe jackfruit kernels 20-25', 'Salt to tast...",41-50,41-50,4,Moderate,Spicy,"[ripe jackfruit kernel, turmeric powder, sambh..."
2,https://www.sanjeevkapoor.com/Recipe/-SABUDANA...,Sabudana Vada,"['Sago (sabudana) ,soaked 1 1/2 cups', 'Chutne...",360.0-420.0,11-15,4,Moderate,Mild,"[sago sabudana, chutney, coconut, coriander le..."
3,https://www.sanjeevkapoor.com/Recipe/PUNJABI-S...,Punjabi Samosa,"['Refined flour (maida) 1 1/2 cups', 'Potatoes...",31-40,41-50,4,Difficult,Tangy,"[refined flour maida, potato, carom seed ajwai..."
4,https://www.sanjeevkapoor.com/Recipe/PANI-PURI...,Pani Puri,"['Puris as required', 'Potatoes , boiled and p...",31-40,0-5,4,Moderate,Tangy,"[puris, potato, moong sprout, roasted cumin po..."


In [24]:
# exporting cleaned data set
df.to_csv('df_cleaned_final.csv', index=False)